In [ ]:
import os
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
my_hf_token = 'XXX'
from huggingface_hub import login
login(token=my_hf_token)

In [ ]:
model_id = "stabilityai/stable-diffusion-3-medium-diffusers"
prompt_text = "a photo of JordanHillsDhalia flower in a vase on a window sill with snow outside"
dimension = 512
#!!! Update the date string in the path to your model, check the stage to see that its there
#'@"MY_FINETUNING_DB"."NOTEBOOKS"."ARTIFACTS"/models//pytorch_lora_weights.safetensors'
path_to_my_model = "@ARTIFACTS/models/SD3Med_MyModel_2024_11_15_161310/pytorch_lora_weights.safetensors"

In [ ]:
get_result = session.file.get(path_to_my_model, "pytorch_lora_weights.safetensors")

In [ ]:
!pip install diffusers

In [ ]:
!pip install bitsandbytes
!pip install -U peft

In [ ]:
import torch
from diffusers import DiffusionPipeline

In [ ]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%Y_%m_%d_%H%M%S")


seed_value=4242
generator=torch.Generator().manual_seed(seed_value)


image_name = "generated_image_sd3_" + dt_string+".png"

pipe_id = model_id
pipe = DiffusionPipeline.from_pretrained(pipe_id, torch_dtype=torch.float16
                                         , text_encoder_3=None, tokenizer_3=None)
pipe.enable_model_cpu_offload()
pipe.load_lora_weights("pytorch_lora_weights.safetensors", adapter_name="trained")
pipe.fuse_lora(adapter_names=["trained"], lora_scale=0.9)



image = pipe(
    prompt=prompt_text,
    negative_prompt="",
    num_inference_steps=28,
    height=512,
    width=512,
    guidance_scale=7.0,
    generator=generator,
).images[0]
image.save(image_name)
put_result = session.file.put(image_name,"@artifacts/output/my_gen_images/", auto_compress= False)
put_result[0].status